In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm, sys
from bs4 import BeautifulSoup
from selenium import webdriver

# scrape normal URL

In [2]:
driver = webdriver.Firefox()

In [3]:
for m in range(1,13):
    with open('linknormal.txt', 'a') as f:
        for d in range(1, 32):
            m, d = str(m), str(d)
            if len(m) == 1:
                m = '0' + m
            if len(d) == 1:
                d = '0' + d
            driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/2019{m}{d}/k100/*')
            time.sleep(10)
            html = driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
            try:
                link_num = int(soup.find('h2', id='query-summary').text.split()[0]) # xxx URLs have been captured...
            except:
                continue
            links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # make a list of URL
            links = [link for link in links if link.endswith('html') or 'html?utm' in link] # only html
            for i in range ((link_num-1) // 50):
                driver.find_element_by_id('resultsUrl_next').click()
                time.sleep(2)
                links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")] # append URL in next page to list
                links = [link for link in links if link.endswith('html') or 'html?utm' in link]
            if links != []:
                f.write('\n'.join(links)+'\n') # append to file

# sort         
with open('linknormal.txt', 'r') as f:
    id_exist = set(pd.read_json('nhkweb.json')['id'].tolist())
    links = f.read().strip().split('\n')
    links = [link for link in links if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
with open('linknormal.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')

In [12]:
# only one day
with open('linknormal.txt', 'a') as f:
    driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/20130127/k100/*')
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    try:
        link_num = int(soup.find('h2', id='query-summary').text.split()[0])
    except:
        sys.exit()
    links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
    links = [link for link in links if link.endswith('html') or 'html?utm' in link]  
    for i in range ((link_num-1) // 50):
        driver.find_element_by_id('resultsUrl_next').click()
        time.sleep(2)
        links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
        links = [link for link in links if link.endswith('html') or 'html?utm' in link]
    if links != []:
        f.write('\n'.join(links)+'\n')
                
with open('linknormal.txt', 'r') as f:
    id_exist = set(pd.read_json('nhkweb.json')['id'].tolist())
    links = f.read().strip().split('\n')
    links = [link for link in links if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
with open('linknormal.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')

# scrape easy URL

In [8]:
with open('linkeasy.txt') as f:
    urls = f.read().strip().split('\n')
    ids = set([x.split('.html')[0].split('/')[-1] for x in urls])

id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
print(len(ids), len(id_exist))
len(ids - id_exist)

1 5757


1

In [4]:
driver = webdriver.Firefox()

In [7]:
with open('linkeasy.txt', 'a') as f:
    for i in range(110, 120):
        driver.get(f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k100{i}/*')
        time.sleep(10)
        html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            link_num = int(soup.find('h2', id='query-summary').text.split()[0])
        except:
            continue
        links = [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
        links = [link for link in links if link.endswith('html') or 'html?utm' in link]  
        for i in range ((link_num-1) // 50):
            driver.find_element_by_id('resultsUrl_next').click()
            time.sleep(2)
            links += [x.a.get('href') for x in soup.tbody.find_all('td', class_="url sorting_1")]
            links = [link for link in links if link.endswith('html') or 'html?utm' in link]
        if links != []:
            f.write('\n'.join(links)+'\n')
            
with open('linkeasy.txt', 'r') as f:
    id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
    links = f.read().strip().split('\n')
    links = [link for link in links if link.split('.html')[0].split('/')[-1] not in id_exist]
    links = sorted(set(links))
with open('linkeasy.txt', 'w') as f:
    f.write('\n'.join(links) + '\n')